In [2]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

In [4]:
experiment_id = "Dop8V030"
round_ids = ["ARN", "R01"]

device = torch.device("cpu")
dtype = torch.float32

In [5]:
sequences = [utils.sequences_from_file(experiment_id, round_id, device) for round_id in round_ids]

In [12]:
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]
freq_single_tuple, freq_pair_tuple, total_reads_tuple = zip(
    *[utils.frequences_from_sequences_oh(seq, pseudo_count=0.0) for seq in sequences_oh])
fi = torch.stack(freq_single_tuple).to(device=device, dtype=dtype)  # n_rounds, L, q
fij = torch.stack(freq_pair_tuple).to(device=device, dtype=dtype)     # n_rounds, L, q, L, q
total_reads = torch.tensor(total_reads_tuple).to(device=device, dtype=dtype)
n_rounds, L, q = fi.size()

In [13]:
k = torch.log(fi[0])
h = torch.log(fi[-1]) - torch.log(fi[0])

tr = tree.Tree()
tr.add_node(-1)

selected_modes = torch.BoolTensor([[1]]).to(device)

Ns0 = energy_models.IndepSites(k)
indep = energy_models.IndepSites(h)
ps = selex_distribution.MultiModeDistribution(indep, normalized=False)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

In [14]:
batch_size = 10**6
data_loaders = [data_loading.SelexRoundDataLoader(seq_oh, batch_size=batch_size) for seq_oh in sequences_oh]
n_rounds = len(data_loaders) 

In [15]:
n_chains = 10**4
chains = training.init_chains(n_rounds, n_chains, L, q, device, dtype)
log_weights = torch.zeros(n_rounds, n_chains, device=device, dtype=dtype)

In [16]:
callbacks = [callback.ConvergenceMetricsCallback()]

In [18]:
n_sweeps = 10
lr = 0.001
target_pearson = 1
max_epochs = 10**3

training.train(model, data_loaders, total_reads, chains, n_sweeps, lr, max_epochs, target_pearson,
               callbacks=callbacks, log_weights=log_weights)

 0.00%[                             ] Epoch: 0/1000 [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
params = indep_sites.init_parameters(fi)
params, history = indep_sites.train(
    fi=fi_all[n],
    total_reads=total_reads_all[n], 
    params=params,
    lr=lr,
    max_epochs=max_epochs,
    target_error=1e-6,
    progress_bar=False)